In [1]:
import json
import os
import cv2
import mediapipe as mp


In [2]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

I0000 00:00:1733094152.306691 16318738 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 76.3), renderer: Apple M2


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733094152.348823 16322680 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733094152.360177 16322679 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [4]:
train_dir = 'train'
train = json.load(open('train/_annotations.coco.json', 'r'))
valid = json.load(open('valid/_annotations.coco.json', 'r'))
test = json.load(open('test/_annotations.coco.json', 'r'))

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
train.keys()

dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])

In [8]:
mp_hands = mp.solutions.hands

# Assuming 'valid' is a dictionary with 'images' and 'annotations'
# valid = {'images': [{'file_name': 'example.jpg'}], 'annotations': [{}]}

with mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5) as hands:
    for img, anno in zip(valid['images'], valid['annotations']):
        # Read the image
        img_src = img['file_name']
        image_path = os.path.join("valid", img_src)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Error reading image: {image_path}")
            continue

        # Convert to RGB for Mediapipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


        # Process the image and detect hands
        results = hands.process(image_rgb)

        # Extract hand keypoints
        keypoints = []
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                keypoints.extend([[h.x, h.y, h.z] for h in hand_landmarks.landmark])
                

        # Update the annotations with detected keypoints
        anno['keypoints'] = keypoints

        flipped_horizontally = cv2.flip(image, 1)
        results = hands.process(flipped_horizontally)

        # Extract hand keypoints
        keypoints = []
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                keypoints.extend([[h.x, h.y, h.z] for h in hand_landmarks.landmark])
                

        # Update the annotations with detected keypoints
        anno['flipped_keypoints'] = keypoints


I0000 00:00:1732721770.951210 3516221 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 76.3), renderer: Apple M2
W0000 00:00:1732721770.979128 3522670 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732721770.991736 3522670 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [9]:
with open("valid/_mediapipe_annotated.json", 'w') as json_file:
    json.dump(valid, json_file, indent=4)
    json_file.flush()

In [6]:
print(train['annotations'][0])
# print(train['annotations'][1])

{'id': 0, 'image_id': 0, 'category_id': 21, 'bbox': [38, 69, 309, 320], 'area': 98880, 'segmentation': [], 'iscrowd': 0}


In [7]:
category = {}
for anno in train['annotations']:
    if anno['category_id'] in category:
        category[anno['category_id']] +=1
    else:
        category[anno['category_id']] =1

In [8]:
category
len(category)

26

In [10]:
sorted(category.keys())

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26]